# Deepfake Detection — Colab-Ready Notebook (with Grad-CAM)

**Author:** Your Name

This Colab-ready notebook downloads the Kaggle dataset `xdxd003/ff-c23`, extracts fixed frames (10 per video) into `data/real/` and `data/fake/`, trains a transfer-learning model, evaluates it, and shows Grad-CAM visualizations.


## 0 — Runtime & Setup (Colab)

In [ ]:
# Install dependencies (run in Colab)
!pip install -q kaggle mtcnn tensorflow==2.11.0 opencv-python-headless==4.6.0.66 scikit-learn matplotlib pillow

import os
os.makedirs('data/real', exist_ok=True)
os.makedirs('data/fake', exist_ok=True)
print('Setup complete. Upload kaggle.json to the runtime or place it in ~/.kaggle/kaggle.json')

## 1 — Download Kaggle dataset `xdxd003/ff-c23`

In [ ]:
# Download the dataset using Kaggle API (Colab)
import os, shutil
kaggle_json_path = '/root/.kaggle/kaggle.json'
if os.path.exists('kaggle.json') and not os.path.exists(kaggle_json_path):
    os.makedirs('/root/.kaggle', exist_ok=True)
    shutil.move('kaggle.json', kaggle_json_path)
    os.chmod(kaggle_json_path, 0o600)

dataset_ref = 'xdxd003/ff-c23'
download_dir = 'kaggle_dataset'
os.makedirs(download_dir, exist_ok=True)
try:
    !kaggle datasets download -d {dataset_ref} -p {download_dir} --unzip
    print('Downloaded dataset to', download_dir)
except Exception as e:
    print('Error downloading via kaggle API. Ensure kaggle.json is uploaded. Error:', e)

## 2 — Inspect dataset structure

In [ ]:
# Quick inspect of dataset
import os
dataset_root = 'kaggle_dataset'
for root, dirs, files in os.walk(dataset_root):
    depth = root[len(dataset_root):].count(os.sep)
    if depth <= 2:
        print(root, '->', len(files), 'files, dirs:', dirs[:6])

## 3 — Extract fixed frames per video into data/real and data/fake

In [ ]:
import cv2, numpy as np, os
from mtcnn.mtcnn import MTCNN
from PIL import Image
from pathlib import Path
detector = MTCNN()
OUT_REAL = 'data/real'
OUT_FAKE = 'data/fake'
os.makedirs(OUT_REAL, exist_ok=True)
os.makedirs(OUT_FAKE, exist_ok=True)

def save_face_from_frame(img, out_path, target_size=(224,224)):
    if img is None:
        return False
    if isinstance(img, np.ndarray):
        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        pil = Image.fromarray(rgb)
    else:
        pil = img.convert('RGB')
    try:
        boxes = detector.detect(pil)
        if boxes is not None and len(boxes[0])>0:
            x1, y1, x2, y2 = boxes[0]
            face = pil.crop((int(x1), int(y1), int(x2), int(y2)))
        else:
            w, h = pil.size; s = min(w,h); left=(w-s)//2; top=(h-s)//2
            face = pil.crop((left, top, left+s, top+s))
    except Exception:
        w, h = pil.size; s=min(w,h); left=(w-s)//2; top=(h-s)//2
        face = pil.crop((left, top, left+s, top+s))
    face = face.resize(target_size)
    face.save(out_path, format='JPEG', quality=90)
    return True

video_exts = ('.mp4', '.avi', '.mov', '.mkv', '.webm')
count_videos = 0
max_videos = 2000
frames_per_video = 10

for root, dirs, files in os.walk('kaggle_dataset'):
    for fname in files:
        if fname.lower().endswith(video_exts):
            fpath = os.path.join(root, fname)
            lower = fpath.lower()
            if 'original' in lower or ('real' in lower and 'manipulated' not in lower):
                out_dir = OUT_REAL
            else:
                out_dir = OUT_FAKE
            try:
                cap = cv2.VideoCapture(fpath)
                if not cap.isOpened():
                    continue
                total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                if total <= 0:
                    cap.release(); continue
                indices = np.linspace(0, max(total-1,0), frames_per_video, dtype=int)
                for i, idx in enumerate(indices):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, int(idx))
                    ret, frame = cap.read()
                    if not ret:
                        continue
                    out_name = os.path.join(out_dir, f"{Path(root).name}_{Path(fname).stem}_{i}.jpg")
                    save_face_from_frame(frame, out_name)
                cap.release()
                count_videos += 1
                if count_videos % 50 == 0:
                    print('Processed videos:', count_videos)
                if count_videos >= max_videos:
                    break
            except Exception as e:
                print('Error processing', fpath, e)
    if count_videos >= max_videos:
        break

print('Done. Extracted frames from', count_videos, 'videos.')

## 4 — Create ImageDataGenerators and verify counts

In [ ]:
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_dir = Path('data')
generator = ImageDataGenerator(validation_split=0.2, rescale=1./255)

train_gen = generator.flow_from_directory(
    data_dir,
    target_size=(224,224),
    batch_size=16,
    class_mode='binary',
    subset='training',
    shuffle=True
)
val_gen = generator.flow_from_directory(
    data_dir,
    target_size=(224,224),
    batch_size=16,
    class_mode='binary',
    subset='validation',
    shuffle=False
)
print('Train samples:', train_gen.samples, 'Validation samples:', val_gen.samples)

## 5 — Build model (Transfer learning)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
IMG_SHAPE = (224,224,3)
try:
    base_model = tf.keras.applications.EfficientNetB0(include_top=False, input_shape=IMG_SHAPE, weights='imagenet')
    print('Using EfficientNetB0')
except Exception:
    base_model = tf.keras.applications.MobileNetV2(include_top=False, input_shape=IMG_SHAPE, weights='imagenet')
    print('Using MobileNetV2')

base_model.trainable = False
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = models.Model(inputs, outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

## 6 — Train (lightweight demo)

In [ ]:
EPOCHS = 2
history = None
if train_gen.samples > 0:
    history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS)
else:
    print('No training samples found. Make sure the extraction step ran successfully and data/real & data/fake have images.')

## 7 — Evaluation: Confusion Matrix & Metrics

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
if val_gen.samples > 0 and history is not None:
    val_gen.reset()
    preds = model.predict(val_gen, verbose=1)
    y_pred = (preds.ravel() > 0.5).astype(int)
    y_true = val_gen.classes
    print('Classification report:\n', classification_report(y_true, y_pred, digits=4))
    print('Confusion matrix:\n', confusion_matrix(y_true, y_pred))
else:
    print('Skipping evaluation — no validation samples or no history')

## 8 — Grad-CAM visualization (explainability)

In [ ]:
import tensorflow as tf, numpy as np, matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image as keras_image
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / (tf.math.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy()

def find_last_conv_layer(model):
    for layer in reversed(model.layers):
        if len(layer.output_shape) == 4:
            return layer.name
    raise ValueError('Could not find 4D layer.')

last_conv = find_last_conv_layer(model)
print('Last conv layer:', last_conv)

def show_gradcam_on_image(img_path, model, last_conv_layer_name):
    import cv2, numpy as np
    img = keras_image.load_img(img_path, target_size=(224,224))
    arr = keras_image.img_to_array(img)/255.0
    inp = np.expand_dims(arr, axis=0)
    preds = model.predict(inp)[0][0]
    heatmap = make_gradcam_heatmap(inp, model, last_conv_layer_name)
    heatmap = cv2.resize(heatmap, (224,224))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    img_rgb = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    img_rgb = cv2.resize(img_rgb, (224,224))
    superimposed = cv2.addWeighted(img_rgb, 0.6, heatmap, 0.4, 0)
    plt.figure(figsize=(6,6))
    plt.imshow(superimposed)
    plt.axis('off')
    plt.title(f'Pred: {preds:.4f} (0=Real,1=Fake)')
    plt.show()

## 9 — Run Grad-CAM on a validation image

In [ ]:
import glob
val_imgs = glob.glob('data/*/*.*')[:20]
if len(val_imgs) == 0:
    print('No images found under data/. Run extraction step.')
else:
    sample = val_imgs[0]
    print('Sample image:', sample)
    show_gradcam_on_image(sample, model, last_conv)

## 10 — Inference helper

In [ ]:
from PIL import Image, ImageOps
def predict_image(img_path, model, threshold=0.5):
    img = Image.open(img_path).convert('RGB').resize((224,224))
    arr = np.array(img)/255.0
    p = model.predict(np.expand_dims(arr, axis=0))[0][0]
    label = 'Fake' if p > threshold else 'Real'
    return label, float(p)

if len(glob.glob('data/*/*.*'))>0:
    print('Example prediction:', predict_image(glob.glob('data/*/*.*')[0], model))

## Notes & Next Steps
- Increase `frames_per_video` and `max_videos` for more data.
- Use Colab GPU for faster training (Runtime -> Change runtime type -> GPU).
- For improved performance, unfreeze backbone layers and fine-tune, or use temporal models (3D CNNs / LSTM).